In [26]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [27]:
def evaluate(actual, pred):
    """Computes the R value of the predictions given the actual values.
    
    Both actual and pred should be of the same length where each index
    corresponds to the actual and predicted value of the same row of data.
    
    Args:
        actual: A list of actual values.
        pred: A list of predicted values.
    
    Returns:
        R value of the predictions given the actual values.
    """
    assert len(actual) == len(pred)
    nume = np.sum(np.square(actual - pred))
    denom = np.sum(np.square(actual - np.mean(actual)))
    r_sq = 1 - nume / denom
    return np.sign(r_sq) * np.sqrt(np.abs(r_sq))

In [182]:
class Model(object):
    """A generic Model interface to encapsulate modeling and data processing.
    
    Attributes:
        model_: A sklearn model with two function interfaces - fit, predict.
        columns_: List of column names corresponding to the features.
    """
    def __init__(self, model):
        """Initialize the Model class with the given model.
        
        Args:
            model: A sklearn model.
        """
        self.model_ = model
        
    def fit(self, df):
        """Fit the underlying model with the given data frame.
        
        Args:
            df: Dataframe object containing the underlying data. It has three
                special columns - id, y (label), timestamp.
        """
        # Sets the NaN values as 0 and drops the two special columns.
        X_train = df.drop(['id', 'y', 'timestamp'], axis=1).fillna(0)
        Y_train = df['y']
        self.model_.fit(X_train, Y_train)
        self.columns_ = X_train.columns
    
    def predict(self, df):
        """Infers the predictions for the given df features.
        
        Args:
            df: Dataframe object containing the data features to be predicted. It has
                two special columns - id, timestamp.
        
        Returns:
            The predictions for each of the row in df, in the same order as given.
        """
        return self.model_.predict(df.drop(['id', 'timestamp'], axis=1).fillna(0))
    
    def visualize(self):
        """Visualize the underlying learned model."""
        pass

In [183]:
class LinearRegressionModel(Model):
    def __init__(self):
        super(LinearRegressionModel, self).__init__(LinearRegression())
    
    def visualize(self):
        assert len(self.columns_) == len(self.model_.coef_), \
            "# columns: %d vs # coeff: %d" % (len(self.columns_), len(self.model_.coef_))
        print('Bias: ', self.model_.intercept_)
        for (ind, column) in enumerate(self.columns_):
            print(column, ': ', self.model_.coef_[ind])

In [126]:
def train_model(train_df):
    """Returns a new model trained using the given data frame.

    Args:
        train_df: DataFrame object corresponding to training data.
    
    Returns:
        Model trained using the data frame.
    """
    model = LinearRegressionModel()
    model.fit(train_df)
    return model

In [39]:
def run_infer(model, features, target):
    """Populates the target using the features and model and updates the
    model.
    
    Args:
        model: A model for prediction and then for updating with new data.
        features: DataFrame object corresponding to new data to predict
                  on, and then update the existing model.
        target: DataFrame object containing id and the y value to be updated
                with.
    """
    # Verify that the features and targets are aligned.
    assert len(features) == len(target)
    assert (features['id'] == target['id']).all()

    # Set the target value to the one given.
    target['y'] = model.predict(features)
    return target

In [184]:
# A cell to perform local E2E training and prediction that simulates
# the KaggleGym behavior.
def localE2E(run_validation=True, visualize_model=False):
    """A function to simulate KaggleGym behavior of predicting per timestep.
    """
    # Here's an example of loading the CSV using Pandas's built-in HDF5 support:
    with pd.HDFStore("train.h5", "r") as train:
        # Note that the "train" dataframe is the only dataframe in the file
        df = train.get("train")
        train_df = df[:806298]
        valid_df = df[806298:]
    
    # NOTE: Training goes here.
    model = train_model(train_df)
    if visualize_model:
        model.visualize()
    
    if not run_validation:
        return 0
    
    # Validation goes here.
    predictions = []
    curr_timestamp = 0
    curr_data = []
    curr_target = []
    for row in valid_df.itertuples():
        # If it is a new timestamp, predict.
        if row.timestamp != curr_timestamp:
            if curr_data:
                # NOTE: Inference and updating goes here.
                new_targets = run_infer(
                    model,
                    # Drop the last column (y).
                    pd.DataFrame(curr_data, columns=valid_df.columns[:-1]),
                    pd.DataFrame(curr_target, columns=['id', 'y']))
                predictions.extend(list(new_targets['y']))
            # Reset arrays.
            curr_data = []
            curr_target= []
            curr_timestamp = row.timestamp
        # Drop the first (index) and last (y) into features.
        curr_data.append(list(row)[1:-1])
        curr_target.append([row.id, 0])

    # Infer for the last timestamp.
    new_targets = run_infer(
        model,
        # Drop the last column (y).
        pd.DataFrame(curr_data, columns=valid_df.columns[:-1]),
        pd.DataFrame(curr_target, columns=['id', 'y']))

    predictions.extend(list(new_targets['y']))
    return evaluate(valid_df['y'], predictions)

print(localE2E(run_validation=False, visualize_model=True)) 

Bias:  0.000255509309195
derived_0 :  -3.79935403107e-27
derived_1 :  -9.33099228834e-19
derived_2 :  7.75481824268e-25
derived_3 :  0.0
derived_4 :  1.51886650068e-25
fundamental_0 :  -6.5455437499e-31
fundamental_1 :  1.59284585172e-23
fundamental_2 :  -4.97555725652e-30
fundamental_3 :  -6.39797066095e-32
fundamental_5 :  1.40149668146e-30
fundamental_6 :  -9.1477904899e-32
fundamental_7 :  -2.83456009951e-28
fundamental_8 :  -2.3649933944e-32
fundamental_9 :  -8.28133242841e-32
fundamental_10 :  3.63901500746e-31
fundamental_11 :  -8.84056011253e-30
fundamental_12 :  8.1227256981e-29
fundamental_13 :  7.38967705777e-31
fundamental_14 :  -3.36369442577e-29
fundamental_15 :  1.28694739008e-30
fundamental_16 :  -1.20542281315e-30
fundamental_17 :  4.46917566637e-21
fundamental_18 :  -8.83249076699e-30
fundamental_19 :  5.48561554009e-31
fundamental_20 :  1.06912396106e-29
fundamental_21 :  6.13253427993e-31
fundamental_22 :  -6.84617313874e-31
fundamental_23 :  -1.38538640913e-28
fund

## Keep track of overall accuracy for each trials.

<pre>

|R value   | Model              |  Normalization | Features | Use Id | Online update |
|----------|--------------------|----------------|----------|--------|----------------
|-0.008351 | Always set to 0    |                |          |        |               |
|-0.005666 | Linear regression  |  None          | All      |   No   |     No        |

</pre>

For Linear regression without any normalization, the coefficient for each feature is very low (around e^-20 - e^-30), probably because the features are not normalized and thus too high. It also seems like one of the weight (derived_3) is 0. Hypothesis is that derived_features is computed from fundamental features.

In [ ]:
# Read data from Kagglegym
import kagglegym

def kaggleE2E():
    """Runs Kaggle training and predictions E2E.
    """
    # Create env.
    env = kagglegym.make()
    # Get first observation.
    observation = env.reset()
    # Get train data.
    train_df = observation.train
    model = train_model(train_df)

    # Predict using Kagglegym.
    while True:
        predictions = run_infer(model, observation.features, observation.target)
        observation, reward, done, info = env.step(predictions)
        # print('Reward: ', reward)
        if done:
            break
    print(info)

kaggleE2E()